---

## Authentication System Database Schema

This database schema is designed to handle user authentication, including features such as email/password signup, social login options, email validation, and password recovery. The schema includes three main tables: `Users`, `Social Logins`, and `Password Resets`.

### 1. **Users Table**

**Purpose**:  
Stores the core credentials and profile information of each user. This is the central entity in the authentication system.

**Fields**:

| Field Name                | Data Type         | Description                                                  |
|---------------------------|-------------------|--------------------------------------------------------------|
| `user_id`                 | INTEGER           | Primary key, auto-incremented unique identifier for each user. |
| `email`                   | VARCHAR(255)      | User's email address, must be unique.                         |
| `password_hash`           | VARCHAR(255)      | Hash of the user's password.                                  |
| `profile_type`            | ENUM('admin', 'user', 'guest') | Type of user profile, with default as 'user'.          |
| `email_verified`          | BOOLEAN           | Boolean flag to indicate if the email address has been verified. |
| `email_verification_token`| VARCHAR(255)      | Token used for the email verification process.                 |
| `created_at`              | DATETIME          | Timestamp when the user account was created.                   |
| `updated_at`              | DATETIME          | Timestamp when the user account was last updated.              |

### 2. **Social Logins Table**

**Purpose**:  
Handles user authentication through social media platforms, linking social media accounts to user profiles in the `Users` table.

**Fields**:

| Field Name                | Data Type         | Description                                                  |
|---------------------------|-------------------|--------------------------------------------------------------|
| `social_id`               | INTEGER           | Primary key, auto-incremented unique identifier for each social login record. |
| `user_id`                 | INTEGER           | Foreign key linking back to the `Users` table.               |
| `provider`                | VARCHAR(50)       | Name of the social media platform (e.g., 'Google', 'Facebook'). |
| `social_user_id`          | VARCHAR(255)      | Unique identifier provided by the social media platform.     |
| `created_at`              | DATETIME          | Timestamp when the social login was created.                  |
| `updated_at`              | DATETIME          | Timestamp when the social login was last updated.             |

### 3. **Password Resets Table**

**Purpose**:  
Manages the process for users to reset their passwords, including storing reset tokens and their expiration times.

**Fields**:

| Field Name                | Data Type         | Description                                                  |
|---------------------------|-------------------|--------------------------------------------------------------|
| `reset_id`                | INTEGER           | Primary key, auto-incremented unique identifier for each password reset record. |
| `user_id`                 | INTEGER           | Foreign key linking back to the `Users` table.               |
| `reset_token`             | VARCHAR(255)      | Token used to authenticate the password reset request.       |
| `expiration_date`         | DATETIME          | Expiry date and time for the reset token.                    |
| `created_at`              | DATETIME          | Timestamp when the reset request was created.                 |
| `used`                    | BOOLEAN           | Boolean flag to indicate whether the reset token has been used. |

---


In [6]:
from sqlalchemy import create_engine, text
import pandas as pd 
from datetime import datetime, timedelta

In [9]:
# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///app.db', echo=True)

In [14]:
# List of table names you want to drop
tables = ['users', 'social_logins', 'password_resets']

# Establishing a connection and dropping each table safely
with engine.connect() as connection:
    for table in tables:
        # Executing the DROP TABLE command for each table
        connection.execute(text(f'DROP TABLE IF EXISTS {table}'))

2024-04-17 13:49:41,067 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 13:49:41,067 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS users
2024-04-17 13:49:41,068 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ()
2024-04-17 13:49:41,073 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS social_logins
2024-04-17 13:49:41,074 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ()
2024-04-17 13:49:41,078 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS password_resets
2024-04-17 13:49:41,079 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ()
2024-04-17 13:49:41,084 INFO sqlalchemy.engine.Engine ROLLBACK


In [17]:
# Define raw SQL queries for table creation using text()
create_users_table = text("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    profile_type DEFAULT 'creator',
    email_verified BOOLEAN NOT NULL DEFAULT 0,
    email_verification_token VARCHAR(255),
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

create_social_logins_table = text("""
CREATE TABLE IF NOT EXISTS social_logins (
    social_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    provider VARCHAR(50) NOT NULL,
    social_user_id VARCHAR(255) NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
""")

create_password_resets_table = text("""
CREATE TABLE IF NOT EXISTS password_resets (
    reset_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    reset_token VARCHAR(255) NOT NULL,
    expiration_date DATETIME NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    used BOOLEAN NOT NULL DEFAULT 0
);
""")

In [18]:
# Execute SQL statements
with engine.connect() as connection:
    connection.execute(create_users_table)
    connection.execute(create_social_logins_table)
    connection.execute(create_password_resets_table)

2024-04-17 13:50:04,999 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 13:50:04,999 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    profile_type DEFAULT 'creator',
    email_verified BOOLEAN NOT NULL DEFAULT 0,
    email_verification_token VARCHAR(255),
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
);

2024-04-17 13:50:05,000 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ()
2024-04-17 13:50:05,004 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS social_logins (
    social_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    provider VARCHAR(50) NOT NULL,
    social_user_id VARCHAR(255) NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(

In [30]:
# Query to list all tables in a SQLite database
list_tables_query = text("SELECT name FROM sqlite_master WHERE type='table';")

# Execute the query
with engine.connect() as connection:
    result = connection.execute(list_tables_query)
    tables = result.fetchall()
    for table in tables:
        print(table[0])

2024-04-16 10:08:55,762 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 10:08:55,764 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table';
2024-04-16 10:08:55,765 INFO sqlalchemy.engine.Engine [generated in 0.00237s] ()
sqlite_sequence
users
social_logins
password_resets
2024-04-16 10:08:55,765 INFO sqlalchemy.engine.Engine ROLLBACK


##  Users email
SET email_verified = 1, email_verification_token = NULL
WHERE user_id = 1; 


-- Assume user_id = 1 is the user to verify

In [177]:
# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///app.db', echo=True)# Create an engine that stores data in the local directory's

In [64]:
import random
import string
from datetime import datetime
import hashlib

In [65]:
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

In [66]:
create_users_table = text("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    profile_type DEFAULT 'creator',
    email_verified BOOLEAN NOT NULL DEFAULT 0,
    email_verification_token VARCHAR(255),
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

In [134]:
# Get the current time
current_time = datetime.now()
# Convert current time to Unix timestamp
unix_timestamp = int(current_time.timestamp())


def generate_verification_token():
    #current_time = datetime.now()
    #unix_timestamp = int(current_time.timestamp())
    token = ''.join(random.choices(string.digits, k=6))
    return token

# Example of generating a token
token = generate_verification_token()
print("Generated Verification Token:", token)


def get_user(email):
    select_query = text("SELECT * FROM users WHERE email = :email")

    with engine.connect() as connection:
        result = connection.execute(select_query, {'email': email}).fetchone()
        if result:
            # Map the tuple to a dictionary
            user_dict = {
                'id': result[0],
                'email': result[1],
                'password_hash': result[2],
                'profile_type': result[3],
                'email_verified': result[4],
                'email_verification_token': result[5],
                'created_at': result[6],
                'updated_at': result[7]
            }
            print(f"User found: {user_dict}")
            return user_dict
        else:
            print("User not found.")
            return None


def add_user(email, password, profile_type='creator'):
    register_query = text("""
        INSERT INTO users (email, password_hash, profile_type, email_verification_token)
        VALUES (:email, :password_hash,:profile_type,:email_verification_token);
    """)
    # Using engine.begin() for auto-commit on successful completion
    with engine.begin() as connection:
        connection.execute(register_query, {"email": email, 
                                            "password_hash":hash_password(password),
                                            "profile_type":profile_type,
                                            "email_verification_token":generate_verification_token()
                                            })
        print(f"User {email} registered successfully.")

def delete_user(email):
    delete_query = text("DELETE FROM users WHERE email = :email")
    with engine.begin() as connection:
        connection.execute(delete_query, {'email': email})
        print(f"User {email} deleted successfully.")



def modify_user(email, **kwargs):
    updates = []
    parameters = {'email': email}  # Start with email because it's used in the WHERE clause

    for key, value in kwargs.items():
        if key == 'password' and value is not None:
            # Assume hash_password is a function that hashes passwords
            value = hash_password(value)  # Hashing the password before updating
            key = 'password_hash'  # Ensure database field for password is correctly targeted

        updates.append(f"{key} = :{key}")
        parameters[key] = value

    if updates:
        update_query = text(f"UPDATE users SET {', '.join(updates)} WHERE email = :email")
        with engine.begin() as connection:
            result = connection.execute(update_query, parameters)
            if result.rowcount > 0:
                print(f"User {email} updated successfully.")
            else:
                print(f"No updates performed. No user found with email {email}.")
    else:
        print("No updates provided.")


In [102]:
delete_user('adham1901@hotmail.com')

2024-04-16 10:52:35,202 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 10:52:35,203 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE email = ?


INFO:sqlalchemy.engine.Engine:DELETE FROM users WHERE email = ?


2024-04-16 10:52:35,204 INFO sqlalchemy.engine.Engine [cached since 106.1s ago] ('adham1901@hotmail.com',)


INFO:sqlalchemy.engine.Engine:[cached since 106.1s ago] ('adham1901@hotmail.com',)


User adham1901@hotmail.com deleted successfully.
2024-04-16 10:52:35,206 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [103]:
add_user('adham1901@hotmail.com', 'mymdp')

2024-04-16 10:52:36,539 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 10:52:36,540 INFO sqlalchemy.engine.Engine 
        INSERT INTO users (email, password_hash, profile_type, email_verification_token)
        VALUES (?, ?,?,?);
    


INFO:sqlalchemy.engine.Engine:
        INSERT INTO users (email, password_hash, profile_type, email_verification_token)
        VALUES (?, ?,?,?);
    


2024-04-16 10:52:36,541 INFO sqlalchemy.engine.Engine [cached since 289.9s ago] ('adham1901@hotmail.com', '7248c6fe50257fce78a4ef2db6257197c790e23181d09e83fe3a2de627d5e47e', 'creator', '365894')


INFO:sqlalchemy.engine.Engine:[cached since 289.9s ago] ('adham1901@hotmail.com', '7248c6fe50257fce78a4ef2db6257197c790e23181d09e83fe3a2de627d5e47e', 'creator', '365894')


User adham1901@hotmail.com registered successfully.
2024-04-16 10:52:36,543 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [198]:
pd.read_sql('SELECT * FROM users',engine)

2024-04-16 14:26:28,605 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 14:26:28,606 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM users")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("SELECT * FROM users")


2024-04-16 14:26:28,607 INFO sqlalchemy.engine.Engine [raw sql] ()


...

2024-04-16 14:26:28,612 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2024-04-16 14:26:28,614 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


,user_id,email,password_hash,profile_type,email_verified,email_verification_token,created_at,updated_at
0,4,adham1901@hotmail.com,e9ed69a6ea08507ea7f2d93985c23588cc4c5bd46b0f99...,admin,0,365894,2024-04-16 08:52:36,2024-04-16 08:52:36


In [106]:
modify_user('adham1901@hotmail.com',new_profile_type='admin')

2024-04-16 10:58:14,906 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 10:58:14,907 INFO sqlalchemy.engine.Engine UPDATE users SET profile_type = ? WHERE email = ?


INFO:sqlalchemy.engine.Engine:UPDATE users SET profile_type = ? WHERE email = ?


2024-04-16 10:58:14,908 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ('admin', 'adham1901@hotmail.com')


INFO:sqlalchemy.engine.Engine:[generated in 0.00099s] ('admin', 'adham1901@hotmail.com')


User adham1901@hotmail.com updated successfully.
2024-04-16 10:58:14,911 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [137]:
get_user('adham1901@hotmail.com')

2024-04-16 11:27:19,848 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 11:27:19,849 INFO sqlalchemy.engine.Engine SELECT * FROM users WHERE email = ?


INFO:sqlalchemy.engine.Engine:SELECT * FROM users WHERE email = ?


2024-04-16 11:27:19,850 INFO sqlalchemy.engine.Engine [cached since 378.2s ago] ('adham1901@hotmail.com',)


INFO:sqlalchemy.engine.Engine:[cached since 378.2s ago] ('adham1901@hotmail.com',)


User found: {'id': 4, 'email': 'adham1901@hotmail.com', 'password_hash': '7248c6fe50257fce78a4ef2db6257197c790e23181d09e83fe3a2de627d5e47e', 'profile_type': 'admin', 'email_verified': 0, 'email_verification_token': '365894', 'created_at': '2024-04-16 08:52:36', 'updated_at': '2024-04-16 08:52:36'}
2024-04-16 11:27:19,852 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


{'id': 4,
 'email': 'adham1901@hotmail.com',
 'password_hash': '7248c6fe50257fce78a4ef2db6257197c790e23181d09e83fe3a2de627d5e47e',
 'profile_type': 'admin',
 'email_verified': 0,
 'email_verification_token': '365894',
 'created_at': '2024-04-16 08:52:36',
 'updated_at': '2024-04-16 08:52:36'}

## Password Reset 

In [ ]:
create_password_resets_table = text("""
CREATE TABLE IF NOT EXISTS password_resets (
    reset_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    reset_token VARCHAR(255) NOT NULL,
    expiration_date DATETIME NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    used BOOLEAN NOT NULL DEFAULT 0
);
""")

In [197]:
pd.read_sql('SELECT * FROM password_resets',engine)

2024-04-16 14:26:21,528 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 14:26:21,528 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM password_resets")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("SELECT * FROM password_resets")


2024-04-16 14:26:21,530 INFO sqlalchemy.engine.Engine [raw sql] ()


...

2024-04-16 14:26:21,534 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2024-04-16 14:26:21,536 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


,reset_id,user_id,reset_token,expiration_date,created_at,used
0,1,4,868365,2024-04-17 09:11:06.384067,2024-04-16 09:11:06,0
1,2,4,593876,2024-04-17 09:12:28.214291,2024-04-16 09:12:28,0
2,3,4,573510,2024-04-17 09:12:49.565406,2024-04-16 09:12:49,0
3,4,4,866230,2024-04-17 09:30:07.084317,2024-04-16 09:30:07,0
4,5,4,670889,2024-04-17 09:39:45.189450,2024-04-16 09:39:45,0
5,6,4,813595,2024-04-17 09:39:56.690220,2024-04-16 09:39:56,0
6,7,4,460719,2024-04-17 09:58:43.464949,2024-04-16 09:58:43,0
7,8,4,544553,2024-04-17 10:06:02.891025,2024-04-16 10:06:02,0
8,9,4,688989,2024-04-17 12:19:06.837627,2024-04-16 12:19:06,1


In [217]:
def generate_reset_token(user_id):
    # Generate a random token
    token = generate_verification_token()
    expiration_date = datetime.utcnow() + timedelta(hours=24)  # Token valid for 24 hours

    insert_token = text("""
        INSERT INTO password_resets (user_id, reset_token, expiration_date, used)
        VALUES (:user_id, :reset_token, :expiration_date, 0);
    """)

    with engine.begin() as connection:
        connection.execute(insert_token, {
            "user_id": user_id,
            "reset_token": token,
            "expiration_date": expiration_date
        })
    return token 


# first step to reset the password
def send_reset_email(email):
    user_id = get_user(email)['id']
    token = generate_reset_token(user_id)

    print(f'Send email to {email}  with reset token {token}')


def reset_password(email, token,new_password):
    user_id = get_user(email)['id']
    
    verify_token = text("""
        SELECT * FROM password_resets
        WHERE user_id = :user_id AND reset_token = :reset_token
          AND expiration_date > :current_time AND used = 0;
    """)
    
    current_time = datetime.utcnow()

    with engine.begin() as connection:
        result = connection.execute(verify_token, {
            "user_id": user_id,
            "reset_token": token,
            "current_time": current_time
        }).fetchone()
        if result:
                # Token is valid, reset the password
                update_token_used = text("""
                    UPDATE password_resets
                    SET used = 1
                    WHERE reset_id = :reset_id;
                """)
                
                # Modify 
                modify_user(email=email,new_password=new_password)
                
                connection.execute(update_token_used, {"reset_id": result[0]})
                print("Password has been reset successfully.")
        else:
            print("Invalid or expired reset token.") 


In [202]:
df1 = pd.read_sql('SELECT * FROM users',engine)

2024-04-16 14:28:52,659 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 14:28:52,660 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM users")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("SELECT * FROM users")


2024-04-16 14:28:52,660 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:SELECT * FROM users


2024-04-16 14:28:52,665 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2024-04-16 14:28:52,667 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [203]:
send_reset_email('adham1901@hotmail.com')

2024-04-16 14:29:14,874 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 14:29:14,876 INFO sqlalchemy.engine.Engine SELECT * FROM users WHERE email = ?


INFO:sqlalchemy.engine.Engine:SELECT * FROM users WHERE email = ?


2024-04-16 14:29:14,877 INFO sqlalchemy.engine.Engine [cached since 8552s ago] ('adham1901@hotmail.com',)


...

2024-04-16 14:29:14,881 INFO sqlalchemy.engine.Engine [cached since 608s ago] (4, '801859', datetime.datetime(2024, 4, 17, 12, 29, 14, 879689))


INFO:sqlalchemy.engine.Engine:[cached since 608s ago] (4, '801859', datetime.datetime(2024, 4, 17, 12, 29, 14, 879689))


2024-04-16 14:29:14,883 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


Send email to adham1901@hotmail.com  with reset token 801859


In [204]:
result = reset_password(email='adham1901@hotmail.com',token='801859',new_password='amingugue')

2024-04-16 14:29:27,694 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 14:29:27,696 INFO sqlalchemy.engine.Engine SELECT * FROM users WHERE email = ?


INFO:sqlalchemy.engine.Engine:SELECT * FROM users WHERE email = ?


2024-04-16 14:29:27,697 INFO sqlalchemy.engine.Engine [cached since 8565s ago] ('adham1901@hotmail.com',)


...

INFO:sqlalchemy.engine.Engine:
                    UPDATE password_resets
                    SET used = 1
                    WHERE reset_id = ?;
                


2024-04-16 14:29:27,716 INFO sqlalchemy.engine.Engine [cached since 8565s ago] (11,)


INFO:sqlalchemy.engine.Engine:[cached since 8565s ago] (11,)


Password has been reset successfully.
2024-04-16 14:29:27,719 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [206]:
df2 = pd.read_sql('SELECT * FROM users',engine)

2024-04-16 14:29:35,147 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 14:29:35,148 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM users")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("SELECT * FROM users")


2024-04-16 14:29:35,149 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:SELECT * FROM users


2024-04-16 14:29:35,153 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2024-04-16 14:29:35,155 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [207]:
df2

,user_id,email,password_hash,profile_type,email_verified,email_verification_token,created_at,updated_at
0,4,adham1901@hotmail.com,20db5a9423f0552c251f2a27c0023b6bc4b296e742c90b...,admin,0,365894,2024-04-16 08:52:36,2024-04-16 08:52:36


In [208]:
df1

,user_id,email,password_hash,profile_type,email_verified,email_verification_token,created_at,updated_at
0,4,adham1901@hotmail.com,b22951fb620b9adab45bc8b55e6b71f7962d11abb2a043...,admin,0,365894,2024-04-16 08:52:36,2024-04-16 08:52:36


## Social Logins

In [213]:
print(create_social_logins_table)


CREATE TABLE IF NOT EXISTS social_logins (
    social_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    provider VARCHAR(50) NOT NULL,
    social_user_id VARCHAR(255) NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);



In [209]:
tables

[('sqlite_sequence',), ('users',), ('social_logins',), ('password_resets',)]

In [215]:
get_user('adham1901@hotmail.com')['id']

2024-04-16 14:37:35,436 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 14:37:35,437 INFO sqlalchemy.engine.Engine SELECT * FROM users WHERE email = ?


INFO:sqlalchemy.engine.Engine:SELECT * FROM users WHERE email = ?


2024-04-16 14:37:35,437 INFO sqlalchemy.engine.Engine [cached since 9052s ago] ('adham1901@hotmail.com',)


INFO:sqlalchemy.engine.Engine:[cached since 9052s ago] ('adham1901@hotmail.com',)


User found: {'id': 4, 'email': 'adham1901@hotmail.com', 'password_hash': '20db5a9423f0552c251f2a27c0023b6bc4b296e742c90bcc0936cfcad1cbb182', 'profile_type': 'admin', 'email_verified': 0, 'email_verification_token': '365894', 'created_at': '2024-04-16 08:52:36', 'updated_at': '2024-04-16 08:52:36'}
2024-04-16 14:37:35,439 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


4

In [219]:
def add_social_login(user_id, provider, social_user_id):
    insert_query = text("""
        INSERT INTO social_logins (user_id, provider, social_user_id)
        VALUES (:user_id, :provider, :social_user_id);
    """)
    with engine.begin() as connection:
        connection.execute(insert_query, {
            "user_id": user_id,
            "provider": provider,
            "social_user_id": social_user_id
        })
        print(f"Social login added for user ID {user_id} with provider {provider}.")

In [220]:
# Usage
add_social_login(user_id=1, provider='Google', social_user_id='google123456')

2024-04-16 14:46:25,355 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 14:46:25,356 INFO sqlalchemy.engine.Engine 
        INSERT INTO social_logins (user_id, provider, social_user_id)
        VALUES (?, ?, ?);
    


INFO:sqlalchemy.engine.Engine:
        INSERT INTO social_logins (user_id, provider, social_user_id)
        VALUES (?, ?, ?);
    


2024-04-16 14:46:25,357 INFO sqlalchemy.engine.Engine [generated in 0.00091s] (1, 'Google', 'google123456')


INFO:sqlalchemy.engine.Engine:[generated in 0.00091s] (1, 'Google', 'google123456')


Social login added for user ID 1 with provider Google.
2024-04-16 14:46:25,359 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [221]:
pd.read_sql('SELECT * FROM social_logins',engine)

2024-04-16 14:46:30,844 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-16 14:46:30,844 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM social_logins")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("SELECT * FROM social_logins")


2024-04-16 14:46:30,845 INFO sqlalchemy.engine.Engine [raw sql] ()


...

2024-04-16 14:46:30,849 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2024-04-16 14:46:30,852 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


,social_id,user_id,provider,social_user_id,created_at,updated_at
0,1,1,Google,google123456,2024-04-16 12:46:25,2024-04-16 12:46:25


## Log Table 
- add a session_id 

In [20]:
create_log_table = text('''CREATE TABLE IF NOT EXISTS logs (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    user_id INTEGER,  -- Optional, depending on whether you want to track actions by specific users
    event_type VARCHAR(255),
    description TEXT,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE SET NULL  -- Assumes existence of a users table
);''')

In [21]:
# Execute the command to create the table
with engine.connect() as connection:
    connection.execute(create_log_table)
    print("Logs table created successfully.")

2024-04-17 13:51:21,648 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 13:51:21,648 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS logs (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    user_id INTEGER,  -- Optional, depending on whether you want to track actions by specific users
    event_type VARCHAR(255),
    description TEXT,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE SET NULL  -- Assumes existence of a users table
);
2024-04-17 13:51:21,649 INFO sqlalchemy.engine.Engine [generated in 0.00112s] ()
Logs table created successfully.
2024-04-17 13:51:21,653 INFO sqlalchemy.engine.Engine ROLLBACK


In [24]:
def add_log(user_id, event_type, description=None):
    insert_query = text("""
        INSERT INTO logs (user_id, event_type, description)
        VALUES (:user_id, :event_type, :description);
    """)
    with engine.begin() as connection:
        connection.execute(insert_query, {
            "user_id": user_id,
            "event_type": event_type,
            "description": description
        })
        print(f"Log entry added for event type '{event_type}'.")

def modify_log(log_id, **kwargs):
    updates = [f"{key} = :{key}" for key in kwargs]
    update_query = text(f"UPDATE logs SET {', '.join(updates)} WHERE log_id = :log_id")
    kwargs['log_id'] = log_id

    with engine.begin() as connection:
        result = connection.execute(update_query, kwargs)
        if result.rowcount > 0:
            print(f"Log entry with ID {log_id} updated successfully.")
        else:
            print(f"No log found with ID {log_id}, nothing updated.")

def delete_log(log_id):
    delete_query = text("DELETE FROM logs WHERE log_id = :log_id")

    with engine.begin() as connection:
        result = connection.execute(delete_query, {'log_id': log_id})
        if result.rowcount > 0:
            print(f"Log entry with ID {log_id} deleted successfully.")
        else:
            print(f"No log entry found with ID {log_id}, nothing deleted.")      


def get_log(log_id):
    select_query = text("SELECT * FROM logs WHERE log_id = :log_id")

    with engine.connect() as connection:
        result = connection.execute(select_query, {'log_id': log_id}).fetchone()
        if result:
            log_dict = {
                'log_id': result[0],
                'timestamp': result[1],
                'user_id': result[2],
                'event_type': result[3],
                'description': result[4]
            }
            print(f"Log entry found: {log_dict}")
            return log_dict
        else:
            print("Log entry not found.")
            return None    

In [29]:
pd.read_sql('SELECT * FROM logs',engine)

2024-04-17 13:52:44,564 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 13:52:44,564 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM logs")
2024-04-17 13:52:44,565 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 13:52:44,565 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM logs")
2024-04-17 13:52:44,566 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 13:52:44,566 INFO sqlalchemy.engine.Engine SELECT * FROM logs
2024-04-17 13:52:44,566 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 13:52:44,568 INFO sqlalchemy.engine.Engine ROLLBACK


,log_id,timestamp,user_id,event_type,description
0,1,2024-04-17 11:52:14,1,logIN,None
1,2,2024-04-17 11:52:41,2,logOUT,None


In [26]:
add_log(1,"logIN")

2024-04-17 13:52:14,729 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 13:52:14,730 INFO sqlalchemy.engine.Engine 
        INSERT INTO logs (user_id, event_type, description)
        VALUES (?, ?, ?);
    
2024-04-17 13:52:14,731 INFO sqlalchemy.engine.Engine [generated in 0.00066s] (1, 'logIN', None)
Log entry added for event type 'logIN'.
2024-04-17 13:52:14,732 INFO sqlalchemy.engine.Engine COMMIT


In [28]:
add_log(2,"logOUT")

2024-04-17 13:52:41,942 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 13:52:41,942 INFO sqlalchemy.engine.Engine 
        INSERT INTO logs (user_id, event_type, description)
        VALUES (?, ?, ?);
    
2024-04-17 13:52:41,942 INFO sqlalchemy.engine.Engine [cached since 27.21s ago] (2, 'logOUT', None)
Log entry added for event type 'logOUT'.
2024-04-17 13:52:41,943 INFO sqlalchemy.engine.Engine COMMIT
